<a href="https://colab.research.google.com/github/WhistlingRufus/DeepPavlov_tweets/blob/main/deepPavlov_tweets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##DEEEP PAVLOV RUbert Tutorial


#Скачиваем и импортируем библиотеки

In [ ]:
!pip install tensorflow-gpu==1.15.2
!pip install pandas==1.1.3

!pip install deeppavlov
!python -m deeppavlov install squad_bert
!python -m deeppavlov download paraphraser_rubert

     |████████████████████████████████| 410.9 MB 16 kB/s 
     |████████████████████████████████| 3.8 MB 23.8 MB/s 
     |████████████████████████████████| 503 kB 42.2 MB/s 
     |████████████████████████████████| 50 kB 6.7 MB/s 
  Created wheel for gast: filename=gast-0.2.2-py3-none-any.whl size=7554 sha256=5a57832ab2443b3838ec0c86d5592c59b38d08d9aa52c5c2625a33ad1e6011b8
  Stored in directory: /root/.cache/pip/wheels/21/7f/02/420f32a803f7d0967b48dd823da3f558c5166991bfd204eef3
Successfully built gast
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.6.0
    Uninstalling tensorflow-estimator-2.6.0:
      Successfully uninstalled tensorflow-estimator-2.6.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.6.0
    Uninstalling tensorboard-2.6.0:
      Successfully uninstalled tensorboard-2.6.0
  Attempting uninstall: gast
    Found existing installation: gast 0.4.0
    Uninstalling gast-0.4.0:
      Succes

     |████████████████████████████████| 929 kB 5.3 MB/s 
     |████████████████████████████████| 2.9 MB 29.3 MB/s 
     |████████████████████████████████| 53 kB 2.3 MB/s 
     |████████████████████████████████| 40 kB 22 kB/s 
     |████████████████████████████████| 20.1 MB 134 kB/s 
     |████████████████████████████████| 43 kB 1.8 MB/s 
     |████████████████████████████████| 57 kB 5.6 MB/s 
     |████████████████████████████████| 6.7 MB 24.4 MB/s 
     |████████████████████████████████| 8.2 MB 47.4 MB/s 
     |████████████████████████████████| 859 kB 47.3 MB/s 
     |████████████████████████████████| 7.3 MB 18.0 MB/s 
     |████████████████████████████████| 43 kB 1.9 MB/s 
     |████████████████████████████████| 65 kB 4.3 MB/s 
     |████████████████████████████████| 10.4 MB 36.1 MB/s 
     |████████████████████████████████| 510 kB 46.7 MB/s 
     |████████████████████████████████| 2.1 MB 35.3 MB/s 
     |████████████████████████████████| 46 kB 4.0 MB/s 
     |███████████████████████

2021-10-13 13:41:27.2 INFO in 'deeppavlov.core.common.file'['file'] at line 32: Interpreting 'squad_bert' as '/usr/local/lib/python3.7/dist-packages/deeppavlov/configs/squad/squad_bert.json'
2021-10-13 13:41:27.832 WARNING in 'deeppavlov.utils.pip_wrapper.pip_wrapper'['pip_wrapper'] at line 34: found tensorflow-gpu installed, so upgrading it instead of tensorflow
     |████████████████████████████████| 411.0 MB 25 kB/s 
  Attempting uninstall: tensorflow-gpu
    Found existing installation: tensorflow-gpu 1.15.2
    Uninstalling tensorflow-gpu-1.15.2:
      Successfully uninstalled tensorflow-gpu-1.15.2
  Cloning https://github.com/deepmipt/bert.git (to revision feat/multi_gpu) to /tmp/pip-req-build-k_mbzndo
  Running command git clone -q https://github.com/deepmipt/bert.git /tmp/pip-req-build-k_mbzndo
  Created wheel for bert-dp: filename=bert_dp-1.0-py3-none-any.whl size=23591 sha256=f17d642849844b6c25916c785175e634f5a759819775db2b577871c4bdd93647
  Stored in directory: /tmp/pip-ephe

In [ ]:
import os
import pandas as pd
from tqdm import tqdm 
import string
import json
import numpy as np
from sklearn.model_selection import train_test_split
from deeppavlov import train_model, configs,evaluate_model
from bert_dp.tokenization import WordpieceTokenizer,FullTokenizer,load_vocab

# Смотрим какие слова входят в словарь

In [ ]:
vocab_path = os.path.expanduser('~')+'/.deeppavlov/downloads/bert_models/rubert_cased_L-12_H-768_A-12_v1/vocab.txt'
with open(vocab_path,'r') as f:
  tokens = f.readlines()
[t for t in tokens if 'unused' not in t][-10:]

['косо\n',
 'лево\n',
 'Бэм\n',
 'кавалерийском\n',
 'хореографом\n',
 'шумом\n',
 'конструкторским\n',
 'Исламским\n',
 'худшем\n',
 'Цем\n']

# Смотрим, как слова преобразуются в токены

In [ ]:

vocab = load_vocab(vocab_path )
tokenizer = WordpieceTokenizer(vocab=vocab)
tokenizer.tokenize("летучий голандец")

['летуч', '##ий', 'гола', '##нд', '##ец']

# Скачиваем датасет твитов из русскоязычного сегмента

In [ ]:
!rm -r ~/.deeppavlov/downloads/tweets_ds
!wget -P ~/.deeppavlov/downloads/tweets_ds  https://www.dropbox.com/s/fnpq3z4bcnoktiv/positive.csv
!wget -P ~/.deeppavlov/downloads/tweets_ds  https://www.dropbox.com/s/r6u59ljhhjdg6j0/negative.csv


rm: cannot remove '/root/.deeppavlov/downloads/tweets_ds': No such file or directory
--2021-10-13 13:47:54--  https://www.dropbox.com/s/fnpq3z4bcnoktiv/positive.csv
Resolving www.dropbox.com (www.dropbox.com)... 162.125.80.18, 2620:100:6018:18::a27d:312
Connecting to www.dropbox.com (www.dropbox.com)|162.125.80.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/fnpq3z4bcnoktiv/positive.csv [following]
--2021-10-13 13:47:55--  https://www.dropbox.com/s/raw/fnpq3z4bcnoktiv/positive.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc9de890c365880f8f0671f75c97.dl.dropboxusercontent.com/cd/0/inline/BX9XvSk_Tade_6atpQMflLVFXakperJNnPXJXMHBNdpGsdWSCdvwJQirp5NRN4AQuMWsKiMdaMqGNiKWFGbOpCm371Hm18fR0_Kt8ekKQPnNTBAOH3XP-5MZo2JNG2Orm5kfHsPjCnXFY7PABz75-Lj4/file# [following]
--2021-10-13 13:47:55--  https://uc9de890c365880f8f0671f75c97.dl.dropboxusercontent.com/cd/0/inline/BX9

In [ ]:
!pip install pandas==1.1.3

  Using cached pandas-1.1.3-cp37-cp37m-manylinux1_x86_64.whl (9.5 MB)
  Attempting uninstall: pandas
    Found existing installation: pandas 0.25.3
    Uninstalling pandas-0.25.3:
      Successfully uninstalled pandas-0.25.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.22.0 which is incompatible.
deeppavlov 0.17.1 requires pandas==0.25.3, but you have pandas 1.1.3 which is incompatible.


# Приводим датасет в формат пригодный для обучения

In [ ]:
data_path = os.path.expanduser('~')+'/.deeppavlov/downloads/tweets_ds/'
train_df = pd.read_csv(os.path.join(data_path,'negative.csv'),sep=';')
train_df.columns  = range(0,12)
train_df = train_df.iloc[:,3:5]
train_df['class'] = ['negative']*len(train_df)
train_df2 = pd.read_csv(os.path.join(data_path,'positive.csv'),sep=';')
train_df2.columns  = range(0,12)
train_df2 = train_df2.iloc[:,3:5]
train_df2['class'] = ['positive']*len(train_df2)
train_df = pd.concat([train_df,train_df2],axis =0)
train_df.drop(4,axis =1, inplace = True)
train_df.columns  = ['text','label']
train_df.tail()

,text,label
114905,"Спала в родительском доме, на своей кровати......",positive
114906,RT @jebesilofyt: Эх... Мы немного решили сокра...,positive
114907,"Что происходит со мной, когда в эфире #proacti...",positive
114908,"""Любимая,я подарю тебе эту звезду..."" Имя како...",positive
114909,@Ma_che_rie посмотри #непытайтесьпокинутьомск ...,positive


In [ ]:
train_df.head()

,text,label
0,"Коллеги сидят рубятся в Urban terror, а я из-з...",negative
1,@elina_4post как говорят обещаного три года жд...,negative
2,"Желаю хорошего полёта и удачной посадки,я буду...",negative
3,"Обновил за каким-то лешим surf, теперь не рабо...",negative
4,"Котёнка вчера носик разбила, плакала и расстра...",negative


In [ ]:
for t in tqdm(range(len(train_df))):
  train_df.iloc[t,0] = "".join([i.lower() if i not in string.punctuation+'\n' else ' ' for i in train_df.iloc[t,0] ])

100%|██████████| 226832/226832 [01:13<00:00, 3065.92it/s]


# Делим датасет на трейн и тест

In [ ]:

X_train, X_test = train_test_split(train_df, test_size=0.20, random_state=42,shuffle = True)
#X_train
print(len(X_train), len(X_test))
X_train.to_csv(os.path.join(data_path,'train.csv'))
X_test.to_csv(os.path.join(data_path,'test.csv'))

181465 45367


In [ ]:
X_test

,text,label
87004,rt arinaashotyan mishaburantsev ого это зд...,positive
41691,какие то противные сериалы сейчас смотрю что ...,negative
78184,братко меня кинул уехал в москву до 12,negative
106225,elennevsk ну ты же знаешь что я где то все ...,positive
12615,lena abrashkina да всё хорошо только немног...,negative
...,...,...
57969,бл мой день день прогульщика день обкуриных лю...,positive
71620,ovoschehranenie pakhtusova nastyacoyote ох,negative
85031,samosiyka ещё бы одна мысль о каникулах толь...,positive
93157,такого хренового настроения еще не когда не было,negative


# Копируем конфиг модели и правим для решения конкретной задачи

In [ ]:
!mkdir ~/.deeppavlov/configs
!cp /usr/local/lib/python3.7/dist-packages/deeppavlov/configs/classifiers/rusentiment_bert.json ~/.deeppavlov/configs/rusentiment_bert_v2.json
 

In [ ]:
config_path =os.path.join(os.path.expanduser('~')+'/.deeppavlov/','configs','rusentiment_bert_v2.json')
with open(config_path ,'r') as f:
  conf = json.load(f)
conf['dataset_reader']['test']='test.csv'
conf['dataset_reader']['train']='train.csv'
conf['dataset_reader']['data_path']=data_path
conf["chainer"]['pipe'][0]['vocab_file'] = conf["chainer"]['pipe'][0]['vocab_file'] .replace('multi_cased_L-12_H-768_A-12/vocab.txt',
                                                                                                      'rubert_cased_L-12_H-768_A-12_v1/vocab.txt')
conf["chainer"]['pipe'][3]['bert_config_file']=conf["chainer"]['pipe'][3]['bert_config_file'].replace('multi_cased_L-12_H-768_A-12/bert_config.json',
                                                                                                      'rubert_cased_L-12_H-768_A-12_v1/bert_config.json')
conf["chainer"]['pipe'][3]['pretrained_bert']=conf["chainer"]['pipe'][3]['pretrained_bert'].replace('multi_cased_L-12_H-768_A-12/bert_model.ckpt'
                                                                                                    ,'rubert_cased_L-12_H-768_A-12_v1/bert_model.ckpt')
conf["metadata"]['variables']['MODELS_PATH'] += '/classifiers/mydata_rubert_v1/'
conf["metadata"]['variables']['MODEL_PATH'] =  conf["metadata"]['variables']['MODEL_PATH'].replace('v0','v1')
conf["train"]["epochs"]=5
with open(config_path,'w') as f:
  json.dump(conf,f)

# Тренийруем модель

In [ ]:

with open(config_path,'r') as f:
  conf = json.load(f)
model = train_model(conf, download=False)

2021-10-13 13:49:45.613 WARNING in 'deeppavlov.dataset_readers.basic_classification_reader'['basic_classification_reader'] at line 112: Cannot find /root/.deeppavlov/downloads/tweets_ds/valid.csv file
2021-10-13 13:49:51.29 INFO in 'deeppavlov.dataset_iterators.basic_classification_iterator'['basic_classification_iterator'] at line 74: Splitting field <<train>> to new fields <<['train', 'valid']>>
2021-10-13 13:49:51.106 INFO in 'deeppavlov.core.trainers.fit_trainer'['fit_trainer'] at line 68: NNTrainer got additional init parameters ['pytest_max_batches'] that will be ignored:
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package perluniprops to /root/nltk_data...
[nltk_data]   Unzipping misc/perluniprops.zip.
[nltk_data] Downloading package nonbreaking_prefixes to
[nltk_data]     /root/nltk_d

[nltk_data]   Unzipping corpora/nonbreaking_prefixes.zip.
2021-10-13 13:49:53.722 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 101: [saving vocabulary to /root/.deeppavlov/models/classifiers/mydata_rubert_v1/classifiers/rusentiment_bert_v1/classes.dict]











The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Please use `layer.__call__` method instead.





Instructions for updating:
Use standard file APIs to check for files with this prefix.


2021-10-13 13:50:17.811 INFO in 'deeppavlov.models.bert.bert_classifier'['bert_classifier'] at line 99: [initializing model with Bert from /root/.deeppavlov/downloads/bert_models/rubert_cased_L-12_H-768_A-12_v1/bert_model.ckpt]



INFO:tensorflow:Restoring parameters from /root/.deeppavlov/downloads/bert_models/rubert_cased_L-12_H-768_A-12_v1/bert_model.ckpt


2021-10-13 13:52:43.771 INFO in 'deeppavlov.core.trainers.nn_trainer'['nn_trainer'] at line 199: Initial best f1_weighted of 0.3612


{"valid": {"eval_examples_count": 18147, "metrics": {"f1_weighted": 0.3612, "f1_macro": 0.3551, "accuracy": 0.512, "roc_auc": 0.5473}, "time_spent": "0:02:25", "epochs_done": 0, "batches_seen": 0, "train_examples_seen": 0, "impatience": 0, "patience_limit": 5}}

{"train": {"eval_examples_count": 64, "metrics": {"f1_weighted": 0.8907, "f1_macro": 0.8906, "accuracy": 0.8906, "roc_auc": 0.9003}, "time_spent": "1:06:50", "epochs_done": 1, "batches_seen": 2552, "train_examples_seen": 163318, "learning_rate": 1e-05, "loss": 0.44982632644311016}}


2021-10-13 14:59:30.700 INFO in 'deeppavlov.core.trainers.nn_trainer'['nn_trainer'] at line 207: Improved best f1_weighted from 0.3612 to 0.8154
2021-10-13 14:59:30.702 INFO in 'deeppavlov.core.trainers.nn_trainer'['nn_trainer'] at line 209: Saving model
2021-10-13 14:59:30.703 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 75: [saving model to /root/.deeppavlov/models/classifiers/mydata_rubert_v1/classifiers/rusentiment_bert_v1/model]


{"valid": {"eval_examples_count": 18147, "metrics": {"f1_weighted": 0.8154, "f1_macro": 0.8154, "accuracy": 0.8154, "roc_auc": 0.9005}, "time_spent": "1:09:12", "epochs_done": 1, "batches_seen": 2552, "train_examples_seen": 163318, "impatience": 0, "patience_limit": 5}}

